# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step2 加载数据

In [3]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
data = data.dropna()  # 将空值数据删掉
data

"""
data = data.dropna() 是一个在 Python 编程语言中经常使用的代码，特别是在使用 pandas 库进行数据处理时。

这行代码的意思是：

data: 这是一个变量，通常代表一个 pandas DataFrame 或 Series，也就是你的数据集。

.dropna(): 这是一个方法（method），它的作用是**移除（drop）数据集中所有包含缺失值（missing values）**的行（row）或列（column）。

data = ...: 这一部分是将处理后的结果重新赋值给原始的 data 变量，这意味着原始的、包含缺失值的数据集被新的、不含缺失值的数据集所替代。

简单来说，这行代码的作用就是清理数据，它会从你的数据集中删除所有有空值（例如 NaN, None）的记录，从而保证后续的数据分析或模型训练不会因为缺失数据而中断或出错。

默认情况下，dropna() 会移除任何包含一个或多个缺失值的行。你也可以通过设置参数来改变它的行为，例如：

data.dropna(axis=1)：会移除包含缺失值的列。

data.dropna(how='all')：只会移除整行都是缺失值的行。
"""

"\ndata = data.dropna() 是一个在 Python 编程语言中经常使用的代码，特别是在使用 pandas 库进行数据处理时。\n\n这行代码的意思是：\n\ndata: 这是一个变量，通常代表一个 pandas DataFrame 或 Series，也就是你的数据集。\n\n.dropna(): 这是一个方法（method），它的作用是**移除（drop）数据集中所有包含缺失值（missing values）**的行（row）或列（column）。\n\ndata = ...: 这一部分是将处理后的结果重新赋值给原始的 data 变量，这意味着原始的、包含缺失值的数据集被新的、不含缺失值的数据集所替代。\n\n简单来说，这行代码的作用就是清理数据，它会从你的数据集中删除所有有空值（例如 NaN, None）的记录，从而保证后续的数据分析或模型训练不会因为缺失数据而中断或出错。\n\n默认情况下，dropna() 会移除任何包含一个或多个缺失值的行。你也可以通过设置参数来改变它的行为，例如：\n\ndata.dropna(axis=1)：会移除包含缺失值的列。\n\ndata.dropna(how='all')：只会移除整行都是缺失值的行。\n"

## Step3 创建Dataset

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]

    def __len__(self):
        return len(self.data)


"""
代码详解
__init__(self) -> None:
这是 初始化方法。当创建一个 MyDataset 类的对象时，这个方法会被自动调用。

super().__init__(): 调用父类 Dataset 的构造函数，这是继承类时的标准做法。

self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv"): 使用 pandas 库读取名为 "ChnSentiCorp_htl_all.csv" 的 CSV 文件，并将其内容存储在实例变量 self.data 中。

self.data = self.data.dropna(): 清理数据，移除所有包含缺失值的行，确保数据完整性。

__getitem__(self, index)
这个方法是 Dataset 类的核心，它使得数据集可以像列表一样通过索引来访问。当你通过 dataset[i] 访问数据时，这个方法就会被调用。

self.data.iloc[index]: 使用 pandas 的 iloc 方法，根据整数位置 index 获取数据集中的一行数据。

["review"] 和 ["label"]: 从获取的行中，分别取出 "review" 列和 "label" 列的值。通常，review 代表文本数据，而 label 代表其对应的类别（例如，情感分类中的正面或负面）。

return ...: 返回一个元组，包含文本和标签，这是 PyTorch 模型训练时常见的输入格式。

__len__(self)
这个方法返回数据集的总项目数。PyTorch 的 DataLoader 会调用这个方法来确定数据集的大小，以便进行批处理和迭代。

return len(self.data): 返回 self.data（即处理后的数据集）的行数，表示数据集中有多少个样本。

总结：这个 MyDataset 类实现了数据加载、缺失值清理、按索引获取单个数据样本和获取数据集大小的功能。它是 PyTorch 数据加载管道中的重要组成部分。
"""

'\n代码详解\n__init__(self) -> None:\n这是 初始化方法。当创建一个 MyDataset 类的对象时，这个方法会被自动调用。\n\nsuper().__init__(): 调用父类 Dataset 的构造函数，这是继承类时的标准做法。\n\nself.data = pd.read_csv("./ChnSentiCorp_htl_all.csv"): 使用 pandas 库读取名为 "ChnSentiCorp_htl_all.csv" 的 CSV 文件，并将其内容存储在实例变量 self.data 中。\n\nself.data = self.data.dropna(): 清理数据，移除所有包含缺失值的行，确保数据完整性。\n\n__getitem__(self, index)\n这个方法是 Dataset 类的核心，它使得数据集可以像列表一样通过索引来访问。当你通过 dataset[i] 访问数据时，这个方法就会被调用。\n\nself.data.iloc[index]: 使用 pandas 的 iloc 方法，根据整数位置 index 获取数据集中的一行数据。\n\n["review"] 和 ["label"]: 从获取的行中，分别取出 "review" 列和 "label" 列的值。通常，review 代表文本数据，而 label 代表其对应的类别（例如，情感分类中的正面或负面）。\n\nreturn ...: 返回一个元组，包含文本和标签，这是 PyTorch 模型训练时常见的输入格式。\n\n__len__(self)\n这个方法返回数据集的总项目数。PyTorch 的 DataLoader 会调用这个方法来确定数据集的大小，以便进行批处理和迭代。\n\nreturn len(self.data): 返回 self.data（即处理后的数据集）的行数，表示数据集中有多少个样本。\n\n总结：这个 MyDataset 类实现了数据加载、缺失值清理、按索引获取单个数据样本和获取数据集大小的功能。它是 PyTorch 数据加载管道中的重要组成部分。\n'

In [6]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


## Step4 划分数据集

In [7]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('房间比较大,环境也还满好.服务态度也不错,设施太陈旧.印象不好的就是餐厅,早餐的餐厅地面比较脏,桌椅都比', np.int64(1))
('第一次用新卡订酒店。简评如下：优点：1.免费升级到套房。2.服务态度好。3.地理位置还好。靠近香港路。窗外望出去就是香格里拉大大的“S”。4.能看见一块像三明治一样大的海。5.两个液晶电视。6.MINIBAR比较便宜。7.里外两套独立的空调系统，出风量大。吹衣服效果不错。缺点：（恐怕比较多）1.房间有新装修的味道。2.房间布局设计问题不小。败笔出处可见。居然还有个隐蔽的衣橱在房间的角落里。3.使用宽带要把书桌拉出来才能看见宽带接口。服务员钻进钻出地帮我找。4.没有睡衣供应。5.装修质量太次了。如果是我的家，我一分钱都不会结给装修队。6.服务有问题。入住的时候，冰箱里什么都没有。MINIBAR什么食品都没有。电话服务台才补上。7.液晶电视是那种小的。大概和我的电脑液晶屏幕差不多大。8.洗漱用品比较次。孤零零地被放在宽大的洗手台上。其它空无一物了。9.保险箱不能用。总之，很多酒店常用设备都缺乏。同样是四星酒店。我还是怀念在栈桥边的泛海名人。房内设施齐全，更加温馨。', np.int64(0))
('此酒店硬件不错，但在酒店吃饭的人较少的原因，菜的花样较少，望能进一步提高', np.int64(1))
('大堂、过道、房间都不错，但卫生间实在与挂牌五星不符，没有独立的淋浴房，只有浴缸配浴帘，作为一个新建的酒店相当落伍。整个山庄感觉类似于一个别墅住宅小区，环境还不错，但也没什么过人之处。早餐偏中式的，品种不多，一般般，五一期间，客人太多，要找个位置都不容易。去用晚餐就被告知没位子了。临安安吉这一带好的酒店少得可怜，陆羽山庄似乎只能是不二的选择了。', np.int64(1))
('房间太小,设计不合理,竟然连会客小台桌都没有,我住的房间马桶冲水不畅,卫生间太小,洗裕区域和马桶区域太过狭窄,很不方便,总体来说,说它是四星级是太言过其实了,2星差不多.', np.int64(0))
('这是一家住宿房间设施极差的度假村，我提前在携程网订了一间5号楼的房间，但是，当我入住时前台因为没有房间，竟然说没有接到订单，然后给了6号楼，空调很响，隔音又差，隔壁房间的一举一动都听见，原计划住两天，吓得第二天赶快逃。离开后发现，其实附近的好的宾馆还是不

## Step5 创建Dataloader

In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

"""
下面是每个参数的具体含义：

DataLoader(...): 这是一个 PyTorch 类，用于封装数据集，并提供按批次加载数据的功能。

trainset: 这是你的数据集对象，通常是你自定义的 Dataset 类的实例（比如你之前定义的 MyDataset）。DataLoader 会从这个数据集中提取数据。

batch_size=32: 这设置了每个批次中包含的样本数量。在这里，每个批次会有 32 个训练样本。使用批处理可以提高训练效率，因为 GPU 擅长并行处理。

shuffle=True: 这个参数决定了在每个训练周期（epoch）开始时，是否对数据集进行随机打乱。打乱数据有助于防止模型学习到数据的顺序，从而提高模型的泛化能力。

collate_fn=collate_func: 这是一个非常重要的参数，它指定了一个名为 collate_func 的函数。这个函数的作用是对一个批次中的所有样本进行处理和打包。例如，如果你的数据是长短不一的句子，collate_func 就会负责将它们填充（padding）到相同的长度，以便可以形成一个张量（tensor）批次。
"""

'\n下面是每个参数的具体含义：\n\nDataLoader(...): 这是一个 PyTorch 类，用于封装数据集，并提供按批次加载数据的功能。\n\ntrainset: 这是你的数据集对象，通常是你自定义的 Dataset 类的实例（比如你之前定义的 MyDataset）。DataLoader 会从这个数据集中提取数据。\n\nbatch_size=32: 这设置了每个批次中包含的样本数量。在这里，每个批次会有 32 个训练样本。使用批处理可以提高训练效率，因为 GPU 擅长并行处理。\n\nshuffle=True: 这个参数决定了在每个训练周期（epoch）开始时，是否对数据集进行随机打乱。打乱数据有助于防止模型学习到数据的顺序，从而提高模型的泛化能力。\n\ncollate_fn=collate_func: 这是一个非常重要的参数，它指定了一个名为 collate_func 的函数。这个函数的作用是对一个批次中的所有样本进行处理和打包。例如，如果你的数据是长短不一的句子，collate_func 就会负责将它们填充（padding）到相同的长度，以便可以形成一个张量（tensor）批次。\n'

In [11]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[  101, 11632,  1039,  ...,     0,     0,     0],
        [  101,  1086,  3613,  ...,     0,     0,     0],
        [  101,   857,   749,  ...,     0,     0,     0],
        ...,
        [  101,  2902,  4212,  ...,  3341,   857,   102],
        [  101,  2902,  2382,  ...,     0,     0,     0],
        [  101,  6818,   707,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 

## Step6 创建模型及优化器

In [13]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

"""
torch.cuda.is_available(): 这是一个布尔函数，它会检查你的系统是否安装了 NVIDIA GPU 以及对应的 CUDA 驱动程序。如果条件满足，它会返回 True；否则，返回 False。

model = model.cuda(): 这行代码只有在 if 条件为 True 时才会执行。model.cuda() 的作用是将 PyTorch 模型的所有参数和缓冲区从 CPU 内存转移到 GPU 显存上。将模型放在 GPU 上，可以利用 GPU 强大的并行计算能力，大大加速训练和推理过程。
"""

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\ntorch.cuda.is_available(): 这是一个布尔函数，它会检查你的系统是否安装了 NVIDIA GPU 以及对应的 CUDA 驱动程序。如果条件满足，它会返回 True；否则，返回 False。\n\nmodel = model.cuda(): 这行代码只有在 if 条件为 True 时才会执行。model.cuda() 的作用是将 PyTorch 模型的所有参数和缓冲区从 CPU 内存转移到 GPU 显存上。将模型放在 GPU 上，可以利用 GPU 强大的并行计算能力，大大加速训练和推理过程。\n'

In [14]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [16]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)


"""
pred.long() == batch["labels"].long():

pred: 这是模型的预测结果，通常是一个浮点型（float）张量。

batch["labels"]: 这是每个样本对应的真实标签，也是一个张量。

.long(): 将预测结果和真实标签都转换为长整型（long）。这是为了确保两者的数据类型一致，可以进行精确的比较。

==: 执行逐元素的相等性比较。如果预测值与真实标签相等，结果张量的对应位置为 True；如果不相等，则为 False。

.float():

将上一步得到的布尔（True/False）张量转换为浮点型张量。True 会被转换为 1.0，False 会被转换为 0.0。现在，你得到的是一个由 1.0（正确预测）和 0.0（错误预测）组成的张量。

.sum():

计算张量中所有元素的和。由于张量只包含 1.0 和 0.0，这个和就是预测正确的样本总数。

acc_num += ...:

将计算出的正确预测数量加到 acc_num 变量上。acc_num 通常用来累积在一个或多个批次中正确预测的总数，以便后续计算总体的准确率。
"""


def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()  # 开启训练模式
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()  # 更新模型
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [17]:
train()

ep: 0, global_step: 0, loss: 0.8285971879959106
ep: 0, global_step: 100, loss: 0.37235987186431885
ep: 0, global_step: 200, loss: 0.3822844922542572
ep: 0, acc: 0.8956185579299927
ep: 1, global_step: 300, loss: 0.21711550652980804
ep: 1, global_step: 400, loss: 0.21080389618873596
ep: 1, acc: 0.907216489315033
ep: 2, global_step: 500, loss: 0.18918563425540924
ep: 2, global_step: 600, loss: 0.20952437818050385
ep: 2, acc: 0.907216489315033


## Step9 模型预测

In [18]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [22]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [21]:
pipe(sen)

[{'label': '好评！', 'score': 0.991969645023346}]